# Test LD module

In [1]:
import Base.Threads.@threads

using Chain
using DLMReader
using Distributions
using GLM
using Glob
using InMemoryDatasets
using IterTools
using Random
using StatsBase
using SnpArrays

In [2]:
include("../src/ld.jl")

clump

### 1) get all snps around il7

In [3]:
gwas_file = "/mnt/r6_data/GWAS_MRsuite/input_MRsuite_hypoT4_SM"
chr = 5
tss = 35852695

gwas_data = filereader(gwas_file, 
                       header = [:chr, :pos, :a_effect_out, :a_other_out, :β_out, :se_out, :pval_out],
                       types = Dict( 1=>Int8, 2=>Int32, 3=>String, 4=>String, 
                                     5=>Float64, 6=>Float64, 7=>Float64),
                       delimiter = '\t', skipto = 2)

,chr,pos,a_effect_out,a_other_out,β_out,se_out,pval_out
,identity,identity,identity,identity,identity,identity,identity
,Int8?,Int32?,String?,String?,Float64?,Float64?,Float64?
1,1,565987,T,C,-0.159782,0.089708,0.07489
2,1,566139,A,C,0.155926,0.132123,0.2379
3,1,568665,A,G,-0.054151,0.057159,0.3434
4,1,592368,A,G,-0.061272,0.063967,0.3381
5,1,662531,A,G,0.04487,0.054755,0.4125
6,1,662613,A,G,-0.03076,0.038043,0.4188
7,1,662622,A,G,-0.005365,0.012785,0.6748
8,1,666249,T,C,0.155111,0.124543,0.213


In [4]:
in_window(s::SubArray) = (s[1] == chr && abs(s[2]-tss)≤500000 && s[3]<1e-8)

filter!(gwas_data, [:chr, :pos, :pval_out], type = in_window)
sort!(gwas_data, :pval_out)

,chr,pos,a_effect_out,a_other_out,β_out,se_out,pval_out
,identity,identity,identity,identity,identity,identity,identity
,Int8?,Int32?,String?,String?,Float64?,Float64?,Float64?
1,5,35846815,T,G,-0.059365,0.008319,9.621e-13
2,5,35837234,A,G,-0.059342,0.00832,9.856e-13
3,5,35841449,A,G,0.059324,0.00832,1.005e-12
4,5,35844125,A,G,-0.059285,0.00832,1.036e-12
5,5,35843832,C,G,0.059281,0.00832,1.038e-12
6,5,35833268,T,C,0.055793,0.00806,4.454e-12
7,5,35803577,T,C,-0.056192,0.008344,1.642e-11
8,5,35833717,A,G,0.054111,0.008138,2.941e-11


### 2) get ld matrix

In [5]:
chr_pos = collect(zip(gwas_data.chr, gwas_data.pos))

40-element Vector{Tuple{Any, Any}}:
 (5, 35846815)
 (5, 35837234)
 (5, 35841449)
 (5, 35844125)
 (5, 35843832)
 (5, 35833268)
 (5, 35803577)
 (5, 35833717)
 (5, 35805208)
 (5, 35806110)
 (5, 35866218)
 (5, 35807455)
 (5, 35814105)
 ⋮
 (5, 35857850)
 (5, 35879095)
 (5, 35861152)
 (5, 35857704)
 (5, 35881376)
 (5, 35862841)
 (5, 35850149)
 (5, 35853319)
 (5, 35852311)
 (5, 35810561)
 (5, 35811282)
 (5, 35851831)

In [6]:
files = ["/home/samat33/CrickData_samat33/PLINK/EUR_biallelic/chr$(i)_CEU.BIALL" for i in 1:22]

GenotypesArr = [SnpData(SnpArrays.datadir(file)) for file in files]

@threads for i in 1:lastindex(GenotypesArr)
    GenotypesArr[i].snp_info.idx = collect(1:size(GenotypesArr[i].snp_info, 1))
    GenotypesArr[i].snp_info.chr_pos = collect(
        zip(parse.(Int8, GenotypesArr[i].snp_info.chromosome), 
        GenotypesArr[i].snp_info.position)
    )
    sort!(GenotypesArr[i].snp_info, :chr_pos)
end

In [ ]:
R2_mat, found_v_b = getLDmat(GenotypesArr[5], chr_pos)

In [ ]:
R2_mat

### 3) Clumping

In [7]:
kept_v_b = clump(GenotypesArr[5], chr_pos)

LoadError: MethodError: no method matching clump(::SnpData, ::Vector{Tuple{Any, Any}})
[0mClosest candidates are:
[0m  clump(::SnpData, [91m::AbstractArray{Tuple{T1, T2}, 1}[39m) where {T1, T2<:Integer} at ~/miniconda3/envs/julia_r/share/julia/dev/MrPainter/src/ld.jl:133
[0m  clump(::SnpData, [91m::AbstractArray{Tuple{T1, T2}, 1}[39m, [91m::Float64[39m) where {T1, T2<:Integer} at ~/miniconda3/envs/julia_r/share/julia/dev/MrPainter/src/ld.jl:133

In [ ]:
sum(kept_v_b)

In [ ]:
sum(R2_mat .< 0.1)

In [ ]:
typeof(chr_pos)

In [ ]:
v::Vector{Tuple{Int, Int}} = chr_pos

In [ ]:
typeof(v)